[<img src="../header.svg">](../index.ipynb)

---

# Application: Simulation of Drug Transport across a Virtual Skin Membrane

### Preliminaries

In [1]:
import modsimtools
from modsimtools import *

['/home/jovyan/content/skin', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/jovyan/ug4/lib/', '/home/jovyan/ug4/bin/plugins/ug4py']


ModuleNotFoundError: No module named 'ug4py'

In [33]:
%%writefile modsimtools.py

import os
env_ug4_root = os.environ['UG4_ROOT']
import sys
sys.path.append(env_ug4_root+'/lib/')
sys.path.append(env_ug4_root+'/bin/plugins/ug4py')

Overwriting modsimtools.py


### Create some widgets

In [34]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

s = widgets.Select(
    options=['skin2d-aniso.ugx', 'skin2d-aniso2.ugx', 'skin2d-aniso3.ugx'],
    value='skin2d-aniso.ugx', rows=1,
    description='Geometry:', disabled=False)

r = widgets.BoundedIntText(
    value=2, min=0, max=5,
    step=1, description='Refinements:',
    disabled=False)

display(s)
display(r)

Select(description='Geometry:', options=('skin2d-aniso.ugx', 'skin2d-aniso2.ugx', 'skin2d-aniso3.ugx'), rows=1…

BoundedIntText(value=2, description='Refinements:', max=5)

## Solve problem

In [35]:
%%writefile -a modsimtools.py

import ug4py as ug4
import pylimex as limex
import pyconvectiondiffusion as cd
import pysuperlu as slu

Appending to modsimtools.py


In [36]:
# Setup:
# defining needed subsets, gird and number of refinements
requiredSubsets = ["LIP", "COR", "BOTTOM_SC", "TOP_SC"] # defining subsets
gridName = s.value  # Grid created with ProMesh
numRefs = r.value   # Number of Refinement steps on given grid

In [37]:
%%writefile -a modsimtools.py

def CreateDomain(gridName, numRefs, requiredSubsets):
    # Choosing a domain object
    # (either 1d, 2d or 3d suffix)
    dom = ug4.Domain2d()

    # Loading the given grid into the domain
    print("Loading Domain {gridName}...")
    ug4.LoadDomain(dom, gridName)
    print("Domain loaded.")
    
    
    # Optional: Refining the grid
    if numRefs > 0:
        print("Refining ...")
        refiner = ug4.GlobalDomainRefiner(dom)
        for i in range(numRefs):
            ug4.TerminateAbortedRun()
            refiner.refine()
            print("Refining step {i} ...")

        print("Refining done")

    # checking if geometry has the needed subsets of the probelm
    sh = dom.subset_handler()
    for e in requiredSubsets:
        if sh.get_subset_index(e) == -1:
            print(f"Domain does not contain subset {e}.")
            sys.exit(1)
    
    return dom

Appending to modsimtools.py


In [38]:
#from modsimtools import CreateDomain
dom = CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain {gridName}...
Domain loaded.
Refining ...
Refining step {i} ...
Refining step {i} ...
Refining done


### Create Approximation space

In [39]:
# Create approximation space which describes the unknowns in the equation
approxSpaceDesc = dict(fct = "u", type = "Lagrange", order = 1)

In [40]:
%%writefile -a modsimtools.py

def CreateApproximationSpace(dom, approxSpaceDesc):
    approxSpace = ug4.ApproximationSpace2d(dom)
    approxSpace.add_fct(approxSpaceDesc["fct"], approxSpaceDesc["type"], approxSpaceDesc["order"])
    approxSpace.init_levels()
    approxSpace.init_top_surface()
    print("Approximation space:")
    approxSpace.print_statistic()
    return approxSpace

Appending to modsimtools.py


In [41]:
approxSpace = CreateApproximationSpace(dom, approxSpaceDesc)

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |       0: LIP |       1: COR | 2: BOTTOM_SC |    3: TOP_SC |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          680 |           32 |          608 |           20 |           20 |
| (lev,    1)    |         2613 |          783 |         1752 |           39 |           39 |
| (lev,    2)    |        10241 |         4367 |         5720 |           77 |           77 |
| (lev,    0, g) |          680 |           32 |          608 |           20 |           20 |
| (lev,    1, g) |         2613 |      

## Create a convection-diffusion-equation
$$\frac{\partial Ku}{\partial t} + \nabla \cdot [-DK \nabla u] = 0$$
Partition coefficients

In [42]:
K={}
K["COR"]=1e-3
K["LIP"]=1.0

Diffusion coefficients

In [43]:
D={}
D["COR"]=1.0
D["LIP"]=1.0

Creating two instances of a convection diffusion equation

In [44]:
%%writefile -a modsimtools.py

def CreateDiffusionElemDisc(fname, subdom, mass_scale, diffusion, reaction):
    elemDisc = cd.ConvectionDiffusionFV12d(fname, subdom)
    elemDisc.set_mass_scale(mass_scale)
    elemDisc.set_diffusion(diffusion)
    elemDisc.set_reaction(reaction)
    return elemDisc

Appending to modsimtools.py


In [45]:
elemDisc={}
elemDisc["COR"] = CreateDiffusionElemDisc("u", "COR", K["COR"], K["COR"]*D["COR"], 0.0)
elemDisc["LIP"] = CreateDiffusionElemDisc("u", "LIP", K["LIP"], K["LIP"]*D["LIP"], 0.0)

### Boundary conditions
ug4 separates the boundary value and the discretization
boundary conditions can be enforced through a post-process (dirichlet).
To init at boundary, the value, function name from the Approximationspace
#and the subset name are needed

In [46]:
dirichletBND = ug4.DirichletBoundary2dCPU1()
dirichletBND.add(1.0, "u", "TOP_SC")
dirichletBND.add(0.0, "u", "BOTTOM_SC")

### Global discretization
Create the discretization object which combines all the separate discretizations into one domain discretization.

In [47]:
domainDisc = ug4.DomainDiscretization2dCPU1(approxSpace)
domainDisc.add(elemDisc["COR"])
domainDisc.add(elemDisc["LIP"])
domainDisc.add(dirichletBND)

## Create  solver

In [48]:
lsolver=ug4.LUCPU1()
# lsolver=slu.SuperLUCPU1()

## Solve transient problem

Solve the transient problem.
Use the Approximationspace to create a vector of unknowns and a vector which contains the right hand side

In [49]:
# Define start time, end time and step size
startTime = 0.0
endTime = 1000.0
dt = 25.0

Create a time discretization (with the theta-scheme).
Requires domain and theta ($\theta$ = 1.0 -> implicit Euler, 
$\theta$ = 0.0 -> explicit Euler )


In [50]:
timeDisc=ug4.ThetaTimeStepCPU1(domainDisc, 1.0) 

Create time integrator (requires solver and step size).

In [51]:
timeInt = limex.LinearTimeIntegrator2dCPU1(timeDisc)
#timeInt = limex.ConstStepLinearTimeIntegrator2dCPU1(timeDisc)
timeInt.set_linear_solver(lsolver)
timeInt.set_time_step(dt)

Create and attach an observer object for vtk output:

In [52]:
vtk=ug4.VTKOutput2d()
obs=limex.VTKOutputObserver2dCPU1("SkinData.vtu", vtk)
timeInt.attach_observer(obs)

Solving the transient problem

In [53]:
usol = ug4.GridFunction2dCPU1(approxSpace)

# Init the vector representing the unknowns with 0 to obtain
# reproducable results
ug4.Interpolate(0.0, usol, "u")

# Exporting the result to a vtu-file
# can be visualized in paraview or with a python extension
ug4.WriteGridFunctionToVTK(usol, "SkinData_t0000")

In [54]:
try:
    timeInt.apply(usol, endTime, usol, startTime)
except Exception as inst:
    print(inst)

+++ Integrating: [0, 1000]
+++ Timestep +++1
+++ Reassemble (t=0, dt=25)

ILUT: please use 'set_ordering_algorithm(..)' in the future

Using ILUT(0) on 10241 x 10241 matrix...
[-------------------------25%----------------------50%----------------------75%----------------------] took 4.39603 s.
+++ Timestep +++2
+++ Timestep +++3
+++ Timestep +++4
+++ Timestep +++5
+++ Timestep +++6
+++ Timestep +++7
+++ Timestep +++8
+++ Timestep +++9
+++ Timestep +++10
+++ Timestep +++11
+++ Timestep +++12
+++ Timestep +++13
+++ Timestep +++14
+++ Timestep +++15
+++ Timestep +++16
+++ Timestep +++17
+++ Timestep +++18
+++ Timestep +++19
+++ Timestep +++20
+++ Timestep +++21
+++ Timestep +++22
+++ Timestep +++23
+++ Timestep +++24
+++ Timestep +++25
+++ Timestep +++26
+++ Timestep +++27
+++ Timestep +++28
+++ Timestep +++29
+++ Timestep +++30
+++ Timestep +++31
+++ Timestep +++32
+++ Timestep +++33
+++ Timestep +++34
+++ Timestep +++35
+++ Timestep +++36
+++ Timestep +++37
+++ Timestep +++38
+++ Timest

## Plotting the result using pyvista

In [28]:
import pyvista
result = pyvista.read('SkinData_t0000.vtu')

print("Pyvista input: ")
print(result)
result.plot(scalars="u", show_edges=True, cmap='coolwarm', jupyter_backend='trame' )
#scalars="node_value", categories=True

Pyvista input: 
UnstructuredGrid (0x29c45fca0)
  N Cells:    10032
  N Points:   10241
  X Bounds:   0.000e+00, 3.010e+01
  Y Bounds:   0.000e+00, 1.760e+01
  Z Bounds:   0.000e+00, 0.000e+00
  N Arrays:   1



Widget(value="<iframe src='http://localhost:62023/index.html?ui=P_0x112b3bbd0_0&reconnect=auto' style='width: …